In [41]:
import os
from typing import List, Dict, Any, Optional, Union
from pydantic import BaseModel
import asyncio
import nest_asyncio
nest_asyncio.apply()

from autorest.config import GenerativeConfig
from autorest.examples import pydantic_model_example, database_model_example
from lib.generators import GeneratorAsync

In [13]:
ROOT_DIR = "/Users/beneverman/Documents/Code/Tekmir/case-management-pipeline/case_management_pipeline/projects/api-hub"
db_models = f"api_hub/database/models.py"
routes_dir = f"api_hub/api/v1/routes/"
db_repos = f"api_hub/database/repositories/"
pydantic_models = f"api_hub/api/models.py"

In [3]:
config = GenerativeConfig(root_dir=ROOT_DIR)

In [4]:
test = config.get_db_models(db_models)

joining relative path to root dir /Users/beneverman/Documents/Code/Tekmir/case-management-pipeline/case_management_pipeline/projects/api-hub


In [23]:
routes = os.listdir(os.path.join(ROOT_DIR, routes_dir)) 
routes = [r for r in routes if not r.startswith("__")]
paths = [os.path.join(ROOT_DIR, routes_dir, route) for route in routes]

In [80]:
class GeneratedPydanticModel(BaseModel):
    table_name: str
    imports: str
    code: str

In [90]:
class PydanticModelGenerator(GeneratorAsync):
    def __init__(self, model: str = None, debug: bool = False):
        super().__init__(
            GeneratedPydanticModel,
            system_prompt="You are designed to take in user instructons and answher the user's request concisely with only code and nothing but code.",
            model=model,
            debug=debug,
        )

    async def generate(
        self, one_or_many_queries: Union[List[str], str], model: str = None
    ):
        def _make_prompt(user_insturctions: str):
            intructions = f"""
            INSTRUCITONS:
            Your job is to generate valid pydantic models according to the newest documentation based on the user's request.
            Always output valid code and nothing but code.
            You output imports separately from the rest of the code.
            Always follow the example provided in the output if you are unsure of anything.

            Formatting Rules:
            - If a field can be missing from the database table without breaking the schema, use the Optional[] type hint in the Pydantic model.

            Import Rules:
            - Always include every necessary import.
            - You can combine imports on the same line if they are from the same module.
            - Import the UUID type as follows 'from uuid import UUID'
            - Import ConfigDict as follows 'from pydantic import ConfigDict'

            """
            example = f"""
            EXAMPLE DB MODEL (Input)

            {database_model_example}
            
            EXAMPLE PYDANTIC MODEL (Output)

            {pydantic_model_example}
            """
            query = f"{intructions}\n{example}\nUSER INSTRUCTIONS:\n{user_insturctions}\nOUTPUT:"
            return query

        def _handle_input(one_or_many_queries: Union[List[str], str]):
            """A function to handle the user input and return the list of prompts to be passed to the generator"""
            if type(one_or_many_queries) == str:
                return [
                    _make_prompt(one_or_many_queries)
                ]  # if user passes a string (one query) (we must still reuturn a list even though its one query)
            elif type(one_or_many_queries == list):
                return [
                    _make_prompt(query) for query in one_or_many_queries
                ]  # if user passes a list of queries
            else:
                raise ValueError(
                    f"Expected str or list, got {type(one_or_many_queries)}"
                )

        queries = _handle_input(one_or_many_queries)
        return await super().generate(queries, model)

In [91]:
model_generator = PydanticModelGenerator(model='gpt-4-turbo', debug=True)
res = await model_generator.generate(test['MedicalInjury'])
res = res[0]

Initializing OpenAI Client...
Model: gpt-4-turbo
Model Var: gpt-4-turbo-preview
Max Tokens: 4096
Input Tokens: 123903
Generating 1 queries...
Task id: 0 in: 12.87s
1 of 1 tasks completed
Total time: 12.87s
Error count: 0


In [92]:
imports, code = res.imports, res.code
total = f"{imports}\n{code}"

print(total)

from pydantic import BaseModel, ConfigDict
from typing import Optional
from uuid import UUID
from datetime import date
from enum import Enum

class DatePrecisionEnum(str, Enum):
    Year = 'Year'
    Month = 'Month'
    Day = 'Day'
    Unknown = 'Unknown'
class MedicalInjurySchema(BaseModel):
    id: UUID
    injury_id: UUID
    onset_date: Optional[date]
    onset_date_precision: Optional[DatePrecisionEnum]
    onset_state: Optional[str]
    is_diagnosed: Optional[bool] = False
    diagnosis_date: Optional[date]
    diagnosis_date_precision: Optional[DatePrecisionEnum]
    diagnosis_state: Optional[str]
    diagnosis_provider_id: Optional[UUID]
    injury_severity: Optional[str]
    medical_injury_sub_type: Optional[str]

    class Config(ConfigDict):
        from_attributes = True # this is the updated way to use the from_orm method
        use_enum_values = True # this is for the Enum values to be used instead of the index


In [93]:
exec(total)

In [94]:
class RegeneratedCode(BaseModel):
    code: str
    cause_of_error: Optional[str] = None

In [95]:
class CodeRegenerator(GeneratorAsync):
    def __init__(self, model: str = None, debug: bool = False):
        super().__init__(
            RegeneratedCode,
            system_prompt="You are designed to take in user instructions and answer the user's request concisely with only code and nothing but code.",
            model=model,
            debug=debug,
        )

    async def generate(
        self, one_or_many_queries: Union[List[str], str], model: str = None
    ):
        def _make_prompt(user_instructions: str):
            instructions = f"""
            INSTRUCTIONS:
            Your job is to take in invalid code, brainstorm why it didn't work, fix it, and output only the fixed code.
            Always output valid code and nothing but code.

            Formatting Rules:
            - Ensure all necessary imports are included.
            - Maintain consistent code formatting and adhere to PEP 8 guidelines.
            - Make sure to provide valid Pydantic models according to the latest documentation.

            Import Rules:
            - Always include every necessary import.
            - You can combine imports on the same line if they are from the same module.
            """
            query = f"{instructions}\nUSER INSTRUCTIONS:\n{user_instructions}\nOUTPUT:"
            return query

        def _handle_input(one_or_many_queries: Union[List[str], str]):
            """A function to handle the user input and return the list of prompts to be passed to the generator."""
            if isinstance(one_or_many_queries, str):
                return [_make_prompt(one_or_many_queries)]
            elif isinstance(one_or_many_queries, list):
                return [_make_prompt(query) for query in one_or_many_queries]
            else:
                raise ValueError(
                    f"Expected str or list, got {type(one_or_many_queries)}"
                )

        queries = _handle_input(one_or_many_queries)
        return await super().generate(queries, model)

class DynamicOutputValidator:
    def __init__(self, code: str, retries: int = 2):
        self.code: List[str] = [code]
        self.retries = retries
        self.generator = CodeRegenerator(model='gpt-4-turbo')

    async def regenerate(self):
        for i in range(self.retries):
            current_best_code = self.code[-1]
            try:
                exec(current_best_code)
                return True
            except Exception as e:
                print(f"Regenerating code... try {i+1} / {self.retries}")
                prompt = f"This error was thrown by the code below:\n{str(e)}\nCode:\n{code}"
                res: List[RegeneratedCode] = await self.generator.generate(prompt)
                new_best_code = res[0].code
                self.code.append(new_best_code)
        return None # we failed to regenerate the code

IndentationError: expected an indented block after function definition on line 58 (731300448.py, line 62)